<a href="https://colab.research.google.com/github/InowaR/colab/blob/main/SVD_method_map.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [73]:
import numpy as np

matrix1 = np.array([
    [0, 0, 0, 0, 0],
    [0, 1, 1, 0, 0],
    [0, 1, 1, 0, 0],
    [0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0]
], dtype=np.float32)

matrix2 = np.array([
    [0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0],
    [0, 0, 1, 1, 0],
    [0, 0, 1, 1, 0],
    [0, 0, 0, 0, 0]
], dtype=np.float32)

def estimate_affine_transform_svd(src, dst):
    """
    Определяет аффинное преобразование между двумя изображениями с помощью SVD
    Возвращает смещение (tx, ty)
    """
    # Находим координаты ненулевых пикселей
    src_points = np.column_stack(np.where(src > 0))
    dst_points = np.column_stack(np.where(dst > 0))

    if len(src_points) < 3 or len(dst_points) < 3:
        return 0, 0

    # Центрируем точки
    src_center = np.mean(src_points, axis=0)
    dst_center = np.mean(dst_points, axis=0)

    src_centered = src_points - src_center
    dst_centered = dst_points - dst_center

    # Вычисляем матрицу ковариации
    H = src_centered.T @ dst_centered

    # SVD разложение
    U, S, Vt = np.linalg.svd(H)

    # Матрица поворота
    R = Vt.T @ U.T

    # Вычисляем смещение
    t = dst_center - R @ src_center

    return t[1], t[0]  # возвращаем (x, y)

# Определяем смещение
shift_x, shift_y = estimate_affine_transform_svd(matrix1, matrix2)
print(f"SVD метод: смещение x={shift_x:.1f}, y={shift_y:.1f}")

SVD метод: смещение x=1.0, y=1.0


In [72]:
import numpy as np

def estimate_rotation_svd_fixed(src, dst):
    """
    Исправленный SVD метод для определения угла поворота
    """
    # Находим координаты ненулевых пикселей
    src_points = np.column_stack(np.where(src > 0))
    dst_points = np.column_stack(np.where(dst > 0))

    # Выравниваем количество точек (берем минимум из двух)
    min_points = min(len(src_points), len(dst_points))
    if min_points < 3:
        return 0

    src_points = src_points[:min_points]
    dst_points = dst_points[:min_points]

    # Центрируем точки
    src_center = np.mean(src_points, axis=0)
    dst_center = np.mean(dst_points, axis=0)

    src_centered = src_points - src_center
    dst_centered = dst_points - dst_center

    # Вычисляем матрицу ковариации
    H = src_centered.T @ dst_centered

    # SVD разложение
    U, S, Vt = np.linalg.svd(H)

    # Матрица поворота
    R = Vt.T @ U.T

    # Убеждаемся, что нет отражения
    if np.linalg.det(R) < 0:
        Vt[-1, :] *= -1
        R = Vt.T @ U.T

    # Извлекаем угол из матрицы поворота
    angle = np.arctan2(R[1, 0], R[0, 0])

    return np.degrees(angle)

# Тестовые данные
matrix1 = np.array([
    [0, 0, 0, 0, 0],
    [0, 1, 1, 1, 0],
    [0, 1, 1, 1, 0],
    [0, 1, 1, 1, 0],
    [0, 0, 0, 0, 0]
], dtype=np.float32)

matrix_rotated = np.array([
    [0, 0, 0, 0, 0],
    [0, 1, 1, 1, 0],
    [0, 1, 1, 1, 0],
    [0, 1, 1, 1, 0],
    [0, 0, 0, 0, 0]
], dtype=np.float32)

angle = estimate_rotation_svd_fixed(matrix1, matrix_rotated)
print(f"SVD метод: угол = {angle:.1f}°")

SVD метод: угол = 0.0°
